In [191]:
random_state = 456

import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import math

from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from tensorflow import keras
from sklearn.metrics import confusion_matrix, classification_report

# Dataset

In [2]:
pd.set_option('display.max_column',None)

In [4]:
# loading dataset
df = pd.read_csv('./datasets/Telco_Customer_Churn.csv')
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [5]:
# checking for null values
df.isnull().sum().sum()

0

In [6]:
# info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


In [7]:
# droping unwanted columns
df2 = df.drop('customerID', axis = 1)

# dropping records not having TotalCharges
df2 = df2.drop(df2[df2.TotalCharges == ' '].index)

df2 = df2.reset_index().drop('index', axis = 1)
df2.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [9]:
obj_cols = df2.select_dtypes('O').columns

binary_cols = []
multiclass_cols = []

for i in obj_cols:
    if len(df[i].unique())>2:
        multiclass_cols.append(i)
    else:
        binary_cols.append(i)

# converting catagorical variables to numerical
gender_map = {'Female':0, 'Male':1}
yes_no_map = {'Yes':1,'No':0}
MultipleLines_map = {'Yes':1,'No':0,'No phone service': -1}
yes_no_n_internetservice_map = {'Yes':1,'No':0,'No internet service': -1}
contract_map = {'Month-to-month':0,'One year':1,'Two year':2}

df3 = df2.copy()

df3.gender = df2.gender.map(gender_map)
df3.MultipleLines = df2.MultipleLines.map(MultipleLines_map)
df3.Contract = df2.Contract.map(contract_map)
df3.TotalCharges  = [float(i) for i in df2.TotalCharges]

for i in binary_cols[1:]:
    df3[i] = df2[i].map(yes_no_map)

for i in multiclass_cols[2:8]:
    df3[i] = df2[i].map(yes_no_n_internetservice_map)

In [10]:
cols = ['InternetService_FiberOptic','InternetService_No','PaymentMethod_CreditCard','PaymentMethod_ElectronicCheck','PaymentMethod_MailedCheck']

df4 = df3.drop(['InternetService','PaymentMethod'],axis = 1)

ohe = OneHotEncoder(sparse=False, drop='first')
dummy_df = pd.DataFrame(ohe.fit_transform(df3[['InternetService','PaymentMethod']]),columns=cols)
df4 = df4.join(dummy_df)
df4.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,MonthlyCharges,TotalCharges,Churn,InternetService_FiberOptic,InternetService_No,PaymentMethod_CreditCard,PaymentMethod_ElectronicCheck,PaymentMethod_MailedCheck
0,0,0,1,0,1,0,-1,0,1,0,0,0,0,0,1,29.85,29.85,0,0.0,0.0,0.0,1.0,0.0
1,1,0,0,0,34,1,0,1,0,1,0,0,0,1,0,56.95,1889.50,0,0.0,0.0,0.0,0.0,1.0
2,1,0,0,0,2,1,0,1,1,0,0,0,0,0,1,53.85,108.15,1,0.0,0.0,0.0,0.0,1.0
3,1,0,0,0,45,0,-1,1,0,1,1,0,0,1,0,42.30,1840.75,0,0.0,0.0,0.0,0.0,0.0
4,0,0,0,0,2,1,0,0,0,0,0,0,0,0,1,70.70,151.65,1,1.0,0.0,0.0,1.0,0.0


In [13]:
df4.Churn.value_counts()

0    5163
1    1869
Name: Churn, dtype: int64

In [15]:
#  We have thrice the people staying than those who are leaving
# The dataset is imbalanced
5163/1869

2.7624398073836276

# Model

In [37]:
def ANN(X_train,y_train,epochs,batch_size,tb_callback):
    model = keras.Sequential([
        keras.layers.Dense(22,input_shape = (22,),activation = 'relu'),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(12,input_shape = (22,),activation = 'relu'),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(22,input_shape = (10,),activation = 'relu'),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(22,input_shape = (10,),activation = 'relu'),
        keras.layers.Dense(1,activation = 'sigmoid')
    ])

    model.compile(
        optimizer='adam',
        loss = 'binary_crossentropy',
        metrics = ['accuracy']
    )
    
    model.fit(X_train,y_train,epochs=epochs,batch_size = batch_size,callbacks=tb_callback)
    
    return model

# Training without resolving imbalance

In [20]:
# train test split

X = df4.drop('Churn', axis = 1)
y = df4.Churn
X_train, X_test, y_train, y_test = train_test_split(X,y,train_size=0.8,random_state=random_state)

# Normalization

scaler = MinMaxScaler()
X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)
X_test_scaled = pd.DataFrame(scaler.transform(X_test), columns=X_train.columns)

In [39]:
# training model
tb_callback = keras.callbacks.TensorBoard(log_dir = './logs/customer_churn_unbalanced')

model1 = ANN(X_train_scaled,y_train,epochs=100,batch_size=1000,tb_callback=tb_callback)

Epoch 1/100
6/6 [==============================] - 0s 35ms/step - loss: 0.6281 - accuracy: 0.7067
Epoch 2/100
6/6 [==============================] - 0s 6ms/step - loss: 0.6135 - accuracy: 0.7250
Epoch 3/100
6/6 [==============================] - 0s 6ms/step - loss: 0.6031 - accuracy: 0.7284
Epoch 4/100
6/6 [==============================] - 0s 6ms/step - loss: 0.5967 - accuracy: 0.7301
Epoch 5/100
6/6 [==============================] - 0s 6ms/step - loss: 0.5896 - accuracy: 0.7323
Epoch 6/100
6/6 [==============================] - 0s 6ms/step - loss: 0.5762 - accuracy: 0.7326
Epoch 7/100
6/6 [==============================] - 0s 6ms/step - loss: 0.5660 - accuracy: 0.7332
Epoch 8/100
6/6 [==============================] - 0s 6ms/step - loss: 0.5596 - accuracy: 0.7340
Epoch 9/100
6/6 [==============================] - 0s 6ms/step - loss: 0.5613 - accuracy: 0.7339
Epoch 10/100
6/6 [==============================] - 0s 6ms/step - loss: 0.5604 - accuracy: 0.7340
Epoch 11/100
6/6 [==========

6/6 [==============================] - 0s 6ms/step - loss: 0.4575 - accuracy: 0.7906
Epoch 82/100
6/6 [==============================] - 0s 6ms/step - loss: 0.4544 - accuracy: 0.7892
Epoch 83/100
6/6 [==============================] - 0s 6ms/step - loss: 0.4500 - accuracy: 0.7918
Epoch 84/100
6/6 [==============================] - 0s 6ms/step - loss: 0.4574 - accuracy: 0.7931
Epoch 85/100
6/6 [==============================] - 0s 5ms/step - loss: 0.4525 - accuracy: 0.7943
Epoch 86/100
6/6 [==============================] - 0s 6ms/step - loss: 0.4558 - accuracy: 0.7881
Epoch 87/100
6/6 [==============================] - 0s 6ms/step - loss: 0.4498 - accuracy: 0.7947
Epoch 88/100
6/6 [==============================] - 0s 6ms/step - loss: 0.4545 - accuracy: 0.7913
Epoch 89/100
6/6 [==============================] - 0s 6ms/step - loss: 0.4508 - accuracy: 0.7918
Epoch 90/100
6/6 [==============================] - 0s 6ms/step - loss: 0.4535 - accuracy: 0.7924
Epoch 91/100
6/6 [===============

In [41]:
y_pred = [1 if i>0.5 else 0 for i in model1.predict(X_test_scaled).reshape((1407,))]
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.86      0.87      0.86      1032
           1       0.63      0.61      0.62       375

    accuracy                           0.80      1407
   macro avg       0.74      0.74      0.74      1407
weighted avg       0.80      0.80      0.80      1407



- accuracy = 0.80
- f1_churn_0 = 0.86
- f1_churn_1 = 0.62 -- matters most, we need to increase its f1 score

# Undersampling Majority class

In [60]:
df4.Churn.value_counts()

0    5163
1    1869
Name: Churn, dtype: int64

In [61]:
df4_churn_0 = df4[df4.Churn == 0].sample(1869) # undersampling randomly
df4_churn_1 = df4[df4.Churn == 1]

In [62]:
df4_churn_0.shape, df4_churn_1.shape

((1869, 23), (1869, 23))

In [63]:
df5 = df4_churn_0.append(df4_churn_1)
df5.shape

<ipython-input-63-ac31217c870a>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df5 = df4_churn_0.append(df4_churn_1)


(3738, 23)

In [64]:
# train test split

X = df5.drop('Churn', axis = 1)
y = df5.Churn
X_train, X_test, y_train, y_test = train_test_split(X,y,train_size=0.8,random_state=random_state)

# Normalization

scaler = MinMaxScaler()
X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)
X_test_scaled = pd.DataFrame(scaler.transform(X_test), columns=X_train.columns)

In [65]:
# training model
tb_callback = keras.callbacks.TensorBoard(log_dir = './logs/customer_churn_undersampling')

model2 = ANN(X_train_scaled,y_train,epochs=100,batch_size=1000,tb_callback=tb_callback)

Epoch 1/100
3/3 [==============================] - 0s 76ms/step - loss: 0.7142 - accuracy: 0.4963
Epoch 2/100
3/3 [==============================] - 0s 8ms/step - loss: 0.7075 - accuracy: 0.5067
Epoch 3/100
3/3 [==============================] - 0s 8ms/step - loss: 0.7043 - accuracy: 0.5040
Epoch 4/100
3/3 [==============================] - 0s 8ms/step - loss: 0.7007 - accuracy: 0.5077
Epoch 5/100
3/3 [==============================] - 0s 7ms/step - loss: 0.6952 - accuracy: 0.5151
Epoch 6/100
3/3 [==============================] - 0s 8ms/step - loss: 0.6933 - accuracy: 0.5237
Epoch 7/100
3/3 [==============================] - 0s 8ms/step - loss: 0.6852 - accuracy: 0.5321
Epoch 8/100
3/3 [==============================] - 0s 8ms/step - loss: 0.6894 - accuracy: 0.5247
Epoch 9/100
3/3 [==============================] - 0s 8ms/step - loss: 0.6842 - accuracy: 0.5231
Epoch 10/100
3/3 [==============================] - 0s 8ms/step - loss: 0.6831 - accuracy: 0.5462
Epoch 11/100
3/3 [==========

3/3 [==============================] - 0s 7ms/step - loss: 0.5541 - accuracy: 0.7224
Epoch 80/100
3/3 [==============================] - 0s 8ms/step - loss: 0.5689 - accuracy: 0.7291
Epoch 81/100
3/3 [==============================] - 0s 8ms/step - loss: 0.5457 - accuracy: 0.7375
Epoch 82/100
3/3 [==============================] - 0s 8ms/step - loss: 0.5500 - accuracy: 0.7361
Epoch 83/100
3/3 [==============================] - 0s 7ms/step - loss: 0.5393 - accuracy: 0.7385
Epoch 84/100
3/3 [==============================] - 0s 18ms/step - loss: 0.5500 - accuracy: 0.7301
Epoch 85/100
3/3 [==============================] - 0s 20ms/step - loss: 0.5437 - accuracy: 0.7334
Epoch 86/100
3/3 [==============================] - 0s 8ms/step - loss: 0.5490 - accuracy: 0.7264
Epoch 87/100
3/3 [==============================] - 0s 7ms/step - loss: 0.5556 - accuracy: 0.7281
Epoch 88/100
3/3 [==============================] - 0s 8ms/step - loss: 0.5353 - accuracy: 0.7378
Epoch 89/100
3/3 [=============

In [66]:
y_pred = [1 if i>0.5 else 0 for i in model2.predict(X_test_scaled).reshape((748,))]
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.79      0.69      0.74       392
           1       0.70      0.80      0.75       356

    accuracy                           0.74       748
   macro avg       0.75      0.74      0.74       748
weighted avg       0.75      0.74      0.74       748



- accuracy = 0.74
- f1_churn_0 = 0.78
- f1_churn_1 = 0.75 -- increase in f1 score by 0.13 in expense of class 0

# Oversampling minority class

In [67]:
df4.Churn.value_counts()

0    5163
1    1869
Name: Churn, dtype: int64

In [68]:
df4_churn_0 = df4[df4.Churn == 0]
df4_churn_1 = df4[df4.Churn == 1].sample(5163, replace=True)  # Oversampling randomly

In [69]:
df4_churn_0.shape, df4_churn_1.shape

((5163, 23), (5163, 23))

In [70]:
df5 = df4_churn_0.append(df4_churn_1)
df5.shape

<ipython-input-70-ac31217c870a>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df5 = df4_churn_0.append(df4_churn_1)


(10326, 23)

In [71]:
# train test split

X = df5.drop('Churn', axis = 1)
y = df5.Churn
X_train, X_test, y_train, y_test = train_test_split(X,y,train_size=0.8,random_state=random_state)

# Normalization

scaler = MinMaxScaler()
X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)
X_test_scaled = pd.DataFrame(scaler.transform(X_test), columns=X_train.columns)

In [72]:
# training model
tb_callback = keras.callbacks.TensorBoard(log_dir = './logs/customer_churn_oversampling')

model3 = ANN(X_train_scaled,y_train,epochs=100,batch_size=1000,tb_callback=tb_callback)

Epoch 1/100
9/9 [==============================] - 0s 29ms/step - loss: 0.6940 - accuracy: 0.5172
Epoch 2/100
9/9 [==============================] - 0s 4ms/step - loss: 0.6808 - accuracy: 0.5392
Epoch 3/100
9/9 [==============================] - 0s 5ms/step - loss: 0.6727 - accuracy: 0.5638
Epoch 4/100
9/9 [==============================] - 0s 4ms/step - loss: 0.6625 - accuracy: 0.5847
Epoch 5/100
9/9 [==============================] - 0s 4ms/step - loss: 0.6607 - accuracy: 0.5937
Epoch 6/100
9/9 [==============================] - 0s 5ms/step - loss: 0.6487 - accuracy: 0.6281
Epoch 7/100
9/9 [==============================] - 0s 4ms/step - loss: 0.6429 - accuracy: 0.6377
Epoch 8/100
9/9 [==============================] - 0s 4ms/step - loss: 0.6311 - accuracy: 0.6490
Epoch 9/100
9/9 [==============================] - 0s 4ms/step - loss: 0.6192 - accuracy: 0.6677
Epoch 10/100
9/9 [==============================] - 0s 4ms/step - loss: 0.6130 - accuracy: 0.6682
Epoch 11/100
9/9 [==========

9/9 [==============================] - 0s 4ms/step - loss: 0.5026 - accuracy: 0.7568
Epoch 82/100
9/9 [==============================] - 0s 4ms/step - loss: 0.4999 - accuracy: 0.7579
Epoch 83/100
9/9 [==============================] - 0s 4ms/step - loss: 0.5056 - accuracy: 0.7604
Epoch 84/100
9/9 [==============================] - 0s 4ms/step - loss: 0.5051 - accuracy: 0.7587
Epoch 85/100
9/9 [==============================] - 0s 4ms/step - loss: 0.5030 - accuracy: 0.7616
Epoch 86/100
9/9 [==============================] - 0s 4ms/step - loss: 0.5068 - accuracy: 0.7607
Epoch 87/100
9/9 [==============================] - 0s 4ms/step - loss: 0.5024 - accuracy: 0.7592
Epoch 88/100
9/9 [==============================] - 0s 4ms/step - loss: 0.5022 - accuracy: 0.7640
Epoch 89/100
9/9 [==============================] - 0s 4ms/step - loss: 0.4966 - accuracy: 0.7594
Epoch 90/100
9/9 [==============================] - 0s 4ms/step - loss: 0.5026 - accuracy: 0.7605
Epoch 91/100
9/9 [===============

In [74]:
y_pred = [1 if i>0.5 else 0 for i in model3.predict(X_test_scaled).reshape((2066,))]
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.78      0.76      0.77      1062
           1       0.75      0.77      0.76      1004

    accuracy                           0.77      2066
   macro avg       0.77      0.77      0.77      2066
weighted avg       0.77      0.77      0.77      2066



- accuracy = 0.77
- f1_churn_0 = 0.77
- f1_churn_1 = 0.76 -- both accuracy and f1 score of class 1 increased

# Oversampling : SMOTE

In [93]:
# importing SMOTE

from imblearn.over_sampling import SMOTE

In [94]:
df4.Churn.value_counts()

0    5163
1    1869
Name: Churn, dtype: int64

In [95]:
X = df4.drop('Churn', axis = 1)
y = df4.Churn

sampling = SMOTE(sampling_strategy='minority')
X_sampled, y_sampled = sampling.fit_sample(X,y)

In [96]:
y_sampled.value_counts()

0    5163
1    5163
Name: Churn, dtype: int64

In [97]:
# train test split

X_train, X_test, y_train, y_test = train_test_split(X_sampled,y_sampled,train_size=0.8,random_state=random_state)

# Normalization

scaler = MinMaxScaler()
X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)
X_test_scaled = pd.DataFrame(scaler.transform(X_test), columns=X_train.columns)

In [98]:
# training model
tb_callback = keras.callbacks.TensorBoard(log_dir = './logs/customer_churn_SMOTE')

model4 = ANN(X_train_scaled,y_train,epochs=100,batch_size=1000,tb_callback=tb_callback)

Epoch 1/100
9/9 [==============================] - 0s 26ms/step - loss: 0.7247 - accuracy: 0.4976
Epoch 2/100
9/9 [==============================] - 0s 5ms/step - loss: 0.7059 - accuracy: 0.5098
Epoch 3/100
9/9 [==============================] - 0s 4ms/step - loss: 0.6955 - accuracy: 0.5299
Epoch 4/100
9/9 [==============================] - 0s 4ms/step - loss: 0.6901 - accuracy: 0.5297
Epoch 5/100
9/9 [==============================] - 0s 4ms/step - loss: 0.6810 - accuracy: 0.5584
Epoch 6/100
9/9 [==============================] - 0s 4ms/step - loss: 0.6752 - accuracy: 0.5781
Epoch 7/100
9/9 [==============================] - 0s 4ms/step - loss: 0.6683 - accuracy: 0.5875
Epoch 8/100
9/9 [==============================] - 0s 4ms/step - loss: 0.6590 - accuracy: 0.6056
Epoch 9/100
9/9 [==============================] - 0s 6ms/step - loss: 0.6506 - accuracy: 0.6241
Epoch 10/100
9/9 [==============================] - 0s 4ms/step - loss: 0.6394 - accuracy: 0.6308
Epoch 11/100
9/9 [==========

9/9 [==============================] - 0s 4ms/step - loss: 0.4586 - accuracy: 0.7900
Epoch 82/100
9/9 [==============================] - 0s 4ms/step - loss: 0.4631 - accuracy: 0.7883
Epoch 83/100
9/9 [==============================] - 0s 4ms/step - loss: 0.4550 - accuracy: 0.7955
Epoch 84/100
9/9 [==============================] - 0s 4ms/step - loss: 0.4582 - accuracy: 0.7938
Epoch 85/100
9/9 [==============================] - 0s 4ms/step - loss: 0.4581 - accuracy: 0.7931
Epoch 86/100
9/9 [==============================] - 0s 4ms/step - loss: 0.4579 - accuracy: 0.7906
Epoch 87/100
9/9 [==============================] - 0s 4ms/step - loss: 0.4600 - accuracy: 0.7924
Epoch 88/100
9/9 [==============================] - 0s 4ms/step - loss: 0.4563 - accuracy: 0.7949
Epoch 89/100
9/9 [==============================] - 0s 4ms/step - loss: 0.4551 - accuracy: 0.7933
Epoch 90/100
9/9 [==============================] - 0s 4ms/step - loss: 0.4569 - accuracy: 0.7914
Epoch 91/100
9/9 [===============

In [83]:
y_pred = [1 if i>0.5 else 0 for i in model4.predict(X_test_scaled).reshape((2066,))]
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.81      0.79      0.80      1024
           1       0.80      0.82      0.81      1042

    accuracy                           0.81      2066
   macro avg       0.81      0.81      0.81      2066
weighted avg       0.81      0.81      0.81      2066



- accuracy = 0.81
- f1_churn_0 = 0.80
- f1_churn_1 = 0.81 -- everyone's f1 score increased

# Ensemble Technique

In [180]:
# Normalization

scaler = MinMaxScaler()
df5 = pd.DataFrame(scaler.fit_transform(df4),columns=df4.columns)
df5.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,MonthlyCharges,TotalCharges,Churn,InternetService_FiberOptic,InternetService_No,PaymentMethod_CreditCard,PaymentMethod_ElectronicCheck,PaymentMethod_MailedCheck
0,0.0,0.0,1.0,0.0,0.000000,0.0,0.0,0.5,1.0,0.5,0.5,0.5,0.5,0.0,1.0,0.115423,0.001275,0.0,0.0,0.0,0.0,1.0,0.0
1,1.0,0.0,0.0,0.0,0.464789,1.0,0.5,1.0,0.5,1.0,0.5,0.5,0.5,0.5,0.0,0.385075,0.215867,0.0,0.0,0.0,0.0,0.0,1.0
2,1.0,0.0,0.0,0.0,0.014085,1.0,0.5,1.0,1.0,0.5,0.5,0.5,0.5,0.0,1.0,0.354229,0.010310,1.0,0.0,0.0,0.0,0.0,1.0
3,1.0,0.0,0.0,0.0,0.619718,0.0,0.0,1.0,0.5,1.0,1.0,0.5,0.5,0.5,0.0,0.239303,0.210241,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.014085,1.0,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.0,1.0,0.521891,0.015330,1.0,1.0,0.0,0.0,1.0,0.0


In [181]:
X = df5.drop('Churn', axis = 1)
y = df5.Churn

# train test split
X_train, X_test, y_train, y_test = train_test_split(X,y,train_size=0.8,random_state=random_state)

In [186]:
df6 = X_train.join(y_train)
df6.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,MonthlyCharges,TotalCharges,InternetService_FiberOptic,InternetService_No,PaymentMethod_CreditCard,PaymentMethod_ElectronicCheck,PaymentMethod_MailedCheck,Churn
3576,0.0,0.0,0.0,0.0,0.028169,1.0,0.5,0.5,0.5,0.5,0.5,1.0,0.5,0.0,1.0,0.619403,0.024642,1.0,0.0,0.0,1.0,0.0,0.0
2646,1.0,0.0,0.0,0.0,0.323944,1.0,0.5,0.5,0.5,0.5,0.5,1.0,1.0,0.0,1.0,0.717413,0.256139,1.0,0.0,0.0,1.0,0.0,1.0
3032,1.0,0.0,0.0,0.0,0.436620,1.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.020896,0.079471,0.0,1.0,1.0,0.0,0.0,0.0
5595,1.0,0.0,0.0,0.0,0.225352,1.0,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.0,1.0,0.522886,0.137111,1.0,0.0,0.0,1.0,0.0,0.0
3087,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.5,0.5,0.5,0.5,0.5,0.5,0.0,1.0,0.068159,0.000727,0.0,0.0,0.0,0.0,1.0,1.0


In [187]:
df6.Churn.value_counts()

0.0    4131
1.0    1494
Name: Churn, dtype: int64

In [188]:
# we can make 3 batches here
4131/1494

2.7650602409638556

In [195]:
def train_model(majority_df,minority_df,target_variable):
    models = []
    batches = majority_df.shape[0]//minority_df.shape[0]
    
    if minority_df.shape[0]*batches < majority_df.shape[0]:
        batches += 1
    
    for i in range(batches):
        df = minority_df.append(majority_df.sample(minority_df.shape[0]))
        model = ANN(df.drop(target_variable,axis = 1),df[target_variable],epochs = 100,batch_size=1000,tb_callback=None)
        models.append(model)
    return models

In [196]:
df6_majority = df6[df6.Churn == 0]
df6_minority = df6[df6.Churn == 1]

models = train_model(df6_majority,df6_minority,'Churn')

<ipython-input-195-6deb74a3808e>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = minority_df.append(majority_df.sample(minority_df.shape[0]))


Epoch 1/100
3/3 [==============================] - 0s 2ms/step - loss: 0.8072 - accuracy: 0.5047
Epoch 2/100
3/3 [==============================] - 0s 2ms/step - loss: 0.7653 - accuracy: 0.5023
Epoch 3/100
3/3 [==============================] - 0s 2ms/step - loss: 0.7445 - accuracy: 0.5020
Epoch 4/100
3/3 [==============================] - 0s 2ms/step - loss: 0.7185 - accuracy: 0.5044
Epoch 5/100
3/3 [==============================] - 0s 2ms/step - loss: 0.7151 - accuracy: 0.5067
Epoch 6/100
3/3 [==============================] - 0s 2ms/step - loss: 0.7049 - accuracy: 0.5117
Epoch 7/100
3/3 [==============================] - 0s 2ms/step - loss: 0.6976 - accuracy: 0.5137
Epoch 8/100
3/3 [==============================] - 0s 2ms/step - loss: 0.6931 - accuracy: 0.5271
Epoch 9/100
3/3 [==============================] - 0s 2ms/step - loss: 0.6947 - accuracy: 0.5248
Epoch 10/100
3/3 [==============================] - 0s 2ms/step - loss: 0.6913 - accuracy: 0.5398
Epoch 11/100
3/3 [===========

3/3 [==============================] - 0s 2ms/step - loss: 0.5821 - accuracy: 0.7105
Epoch 85/100
3/3 [==============================] - 0s 2ms/step - loss: 0.5770 - accuracy: 0.7182
Epoch 86/100
3/3 [==============================] - 0s 2ms/step - loss: 0.5772 - accuracy: 0.7135
Epoch 87/100
3/3 [==============================] - 0s 2ms/step - loss: 0.5724 - accuracy: 0.7159
Epoch 88/100
3/3 [==============================] - 0s 2ms/step - loss: 0.5731 - accuracy: 0.7239
Epoch 89/100
3/3 [==============================] - 0s 2ms/step - loss: 0.5843 - accuracy: 0.7149
Epoch 90/100
3/3 [==============================] - 0s 2ms/step - loss: 0.5732 - accuracy: 0.7162
Epoch 91/100
3/3 [==============================] - 0s 2ms/step - loss: 0.5784 - accuracy: 0.7162
Epoch 92/100
3/3 [==============================] - 0s 2ms/step - loss: 0.5705 - accuracy: 0.7219
Epoch 93/100
3/3 [==============================] - 0s 2ms/step - loss: 0.5625 - accuracy: 0.7323
Epoch 94/100
3/3 [===============

<ipython-input-195-6deb74a3808e>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = minority_df.append(majority_df.sample(minority_df.shape[0]))


3/3 [==============================] - 0s 2ms/step - loss: 0.8138 - accuracy: 0.5020
Epoch 2/100
3/3 [==============================] - 0s 2ms/step - loss: 0.7892 - accuracy: 0.5010
Epoch 3/100
3/3 [==============================] - 0s 2ms/step - loss: 0.7516 - accuracy: 0.4970
Epoch 4/100
3/3 [==============================] - 0s 2ms/step - loss: 0.7397 - accuracy: 0.4823
Epoch 5/100
3/3 [==============================] - 0s 2ms/step - loss: 0.7281 - accuracy: 0.5003
Epoch 6/100
3/3 [==============================] - 0s 2ms/step - loss: 0.7196 - accuracy: 0.5141
Epoch 7/100
3/3 [==============================] - 0s 2ms/step - loss: 0.7204 - accuracy: 0.4883
Epoch 8/100
3/3 [==============================] - 0s 2ms/step - loss: 0.7107 - accuracy: 0.4883
Epoch 9/100
3/3 [==============================] - 0s 2ms/step - loss: 0.7027 - accuracy: 0.5194
Epoch 10/100
3/3 [==============================] - 0s 2ms/step - loss: 0.7091 - accuracy: 0.4823
Epoch 11/100
3/3 [=======================

3/3 [==============================] - 0s 2ms/step - loss: 0.5768 - accuracy: 0.7115
Epoch 85/100
3/3 [==============================] - 0s 2ms/step - loss: 0.5700 - accuracy: 0.7239
Epoch 86/100
3/3 [==============================] - 0s 2ms/step - loss: 0.5790 - accuracy: 0.7105
Epoch 87/100
3/3 [==============================] - 0s 2ms/step - loss: 0.5787 - accuracy: 0.7189
Epoch 88/100
3/3 [==============================] - 0s 2ms/step - loss: 0.5700 - accuracy: 0.7172
Epoch 89/100
3/3 [==============================] - 0s 2ms/step - loss: 0.5720 - accuracy: 0.7202
Epoch 90/100
3/3 [==============================] - 0s 2ms/step - loss: 0.5667 - accuracy: 0.7108
Epoch 91/100
3/3 [==============================] - 0s 2ms/step - loss: 0.5641 - accuracy: 0.7172
Epoch 92/100
3/3 [==============================] - 0s 2ms/step - loss: 0.5584 - accuracy: 0.7226
Epoch 93/100
3/3 [==============================] - 0s 2ms/step - loss: 0.5651 - accuracy: 0.7159
Epoch 94/100
3/3 [===============

<ipython-input-195-6deb74a3808e>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = minority_df.append(majority_df.sample(minority_df.shape[0]))


3/3 [==============================] - 0s 2ms/step - loss: 0.7070 - accuracy: 0.4997
Epoch 2/100
3/3 [==============================] - 0s 2ms/step - loss: 0.7123 - accuracy: 0.4963
Epoch 3/100
3/3 [==============================] - 0s 2ms/step - loss: 0.7028 - accuracy: 0.5067
Epoch 4/100
3/3 [==============================] - 0s 2ms/step - loss: 0.7013 - accuracy: 0.5147
Epoch 5/100
3/3 [==============================] - 0s 2ms/step - loss: 0.7002 - accuracy: 0.5104
Epoch 6/100
3/3 [==============================] - 0s 2ms/step - loss: 0.6965 - accuracy: 0.5191
Epoch 7/100
3/3 [==============================] - 0s 2ms/step - loss: 0.6921 - accuracy: 0.5318
Epoch 8/100
3/3 [==============================] - 0s 2ms/step - loss: 0.6959 - accuracy: 0.5187
Epoch 9/100
3/3 [==============================] - 0s 3ms/step - loss: 0.6909 - accuracy: 0.5321
Epoch 10/100
3/3 [==============================] - 0s 2ms/step - loss: 0.6902 - accuracy: 0.5318
Epoch 11/100
3/3 [=======================

3/3 [==============================] - 0s 2ms/step - loss: 0.5593 - accuracy: 0.7169
Epoch 85/100
3/3 [==============================] - 0s 2ms/step - loss: 0.5549 - accuracy: 0.7269
Epoch 86/100
3/3 [==============================] - 0s 2ms/step - loss: 0.5597 - accuracy: 0.7129
Epoch 87/100
3/3 [==============================] - 0s 9ms/step - loss: 0.5515 - accuracy: 0.7242
Epoch 88/100
3/3 [==============================] - 0s 4ms/step - loss: 0.5511 - accuracy: 0.7189
Epoch 89/100
3/3 [==============================] - 0s 2ms/step - loss: 0.5572 - accuracy: 0.7189
Epoch 90/100
3/3 [==============================] - 0s 2ms/step - loss: 0.5512 - accuracy: 0.7159
Epoch 91/100
3/3 [==============================] - 0s 2ms/step - loss: 0.5622 - accuracy: 0.7192
Epoch 92/100
3/3 [==============================] - 0s 2ms/step - loss: 0.5570 - accuracy: 0.7252
Epoch 93/100
3/3 [==============================] - 0s 3ms/step - loss: 0.5515 - accuracy: 0.7159
Epoch 94/100
3/3 [===============

In [198]:
X_test.shape

(1407, 22)

In [202]:
temp = np.zeros(X_test.shape[0], dtype = 'int64')
for i in models:
    temp += np.array([1 if j>=0.5 else 0 for j in i.predict(X_test).reshape(X_test.shape[0],)])

# considering majority vote
y_pred = [1 if i>1 else 0 for i in temp]

print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

         0.0       0.91      0.74      0.82      1032
         1.0       0.53      0.80      0.63       375

    accuracy                           0.75      1407
   macro avg       0.72      0.77      0.72      1407
weighted avg       0.81      0.75      0.77      1407

